In [ ]:
import requests
import csv
import time

def write_to_csv(data):
    with open('maven_artifacts.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(data)

def get_artifacts():
    url = "https://search.maven.org/solrsearch/select"
    rows = 20
    start = 0

    # Write the headers to the CSV file
    write_to_csv(['groupId', 'artifactId', 'version'])

    while True:
        params = {
            "q": "*:*",
            "rows": rows,
            "start": start,
            "wt": "json"
        }

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
        except requests.exceptions.HTTPError as errh:
            print ("HTTP Error:", errh)
            break
        except requests.exceptions.ConnectionError as errc:
            print ("Error Connecting:", errc)
            break
        except requests.exceptions.Timeout as errt:
            print ("Timeout Error:", errt)
            break
        except requests.exceptions.RequestException as err:
            print ("Something went wrong:", err)
            break

        data = response.json()

        docs = data['response']['docs']

        for doc in docs:
            groupId = doc.get('g', 'N/A')
            artifactId = doc.get('a', 'N/A')
            version = doc.get('latestVersion', 'N/A')
            write_to_csv([groupId, artifactId, version])

        num_found = data['response']['numFound']
        if start + rows >= num_found:
            break

        start += rows
        time.sleep(1)  # To prevent rate limiting

get_artifacts()
